In [2]:
import jax
import equinox as eqx
import diffrax as dfx
import jax.numpy as jnp
import jax.random as jrandom
import jax.scipy as jcp 
import optax
import math
from typing import Callable, List
import jax.nn as jnn
import jax.numpy as jnp
import ipdb
from jax import jit
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import time
import os
from jax import Array
from diffrax.custom_types import PyTree, Scalar, Union
import warnings
from scipy.special import gammaln
import sys
sys.path.append('../')
from copy import deepcopy
warnings.filterwarnings("ignore")
from simulated_system.controller import Controller
from simulated_system.data import *
import pickle
import yaml


In [ ]:
config_path = '../configs/pilot.yaml'
with open(config_path) as f:
    config = yaml.safe_load(f)

key = jrandom.PRNGKey(9)
ts = jnp.linspace(0, config['data']['trial_time'], config['data']['n_timepoints'])
t_stim = config['data']['t_stim']
t_wait = config['data']['t_wait']
t_go = t_stim + t_wait
C = config['data']['control_size']
loader = Sys_Dataloader(100, config, key)
controls, outputs =loader.sample_observations(1)
print(np.unique(jnp.argmax(outputs,axis=-1), return_counts=True))
plotting(ts, controls, outputs, t_stim, t_go)